In [287]:
import urllib.request, json
import numpy as np
import pandas as pd

In [288]:
df_sose = pd.read_csv('./data/opencivitas-spesa-storica+dati-irpef.csv')

In [327]:
df_sose.shape

(26702, 98)

In [344]:
df_sose.ANNO.value_counts()

2011    6699
2010    6699
2012    6664
2013    6640
Name: ANNO, dtype: int64

In [329]:
copy = df_sose.dropna()
copy.shape

(0, 98)

In [336]:
columns = ['COMUNE_CAT_COD','Denominazione Comune', 'ANNO', 'Sigla Provincia_y']

In [337]:
sample = df_sose[columns].iloc[:10,:]

In [339]:
sample

,COMUNE_CAT_COD,Denominazione Comune,ANNO,Sigla Provincia_y
0,A001,ABANO TERME,2010,PD
1,A004,ABBADIA CERRETO,2010,LO
2,A005,ABBADIA LARIANA,2010,LC
3,A006,ABBADIA SAN SALVATORE,2010,SI
4,A008,ABBATEGGIO,2010,PE
5,A010,ABBIATEGRASSO,2010,MI
6,A012,ABETONE,2010,PT
7,A013,ABRIOLA,2010,PZ
8,A015,ACCADIA,2010,FG
9,A016,ACCEGLIO,2010,CN


In [342]:
data_types = ['entrate', 'spese']
base_url = 'https://www.openbilanci.it/armonizzati/bilanci/%s-comune-%s/%s/dettaglio.json?year=2016&type=preventivo'

In [343]:
for r in sample.values:
    cod, city, year, province = r
    if city is not 'Nan':
        city = city.lower().replace(' ', '-')
        province = province.lower()
        city_url = base_url  % (city, province, data_types[0])
        print(city_url)

https://www.openbilanci.it/armonizzati/bilanci/abano-terme-comune-pd/entrate/dettaglio.json?year=2016&type=preventivo
https://www.openbilanci.it/armonizzati/bilanci/abbadia-cerreto-comune-lo/entrate/dettaglio.json?year=2016&type=preventivo
https://www.openbilanci.it/armonizzati/bilanci/abbadia-lariana-comune-lc/entrate/dettaglio.json?year=2016&type=preventivo
https://www.openbilanci.it/armonizzati/bilanci/abbadia-san-salvatore-comune-si/entrate/dettaglio.json?year=2016&type=preventivo
https://www.openbilanci.it/armonizzati/bilanci/abbateggio-comune-pe/entrate/dettaglio.json?year=2016&type=preventivo
https://www.openbilanci.it/armonizzati/bilanci/abbiategrasso-comune-mi/entrate/dettaglio.json?year=2016&type=preventivo
https://www.openbilanci.it/armonizzati/bilanci/abetone-comune-pt/entrate/dettaglio.json?year=2016&type=preventivo
https://www.openbilanci.it/armonizzati/bilanci/abriola-comune-pz/entrate/dettaglio.json?year=2016&type=preventivo
https://www.openbilanci.it/armonizzati/bilanc

In [324]:
city_url = base_url  % (city, province, data_types[0])
with urllib.request.urlopen(city_url) as url:
    data = json.loads(url.read().decode())

HTTPError: HTTP Error 404: Not Found

In [22]:
data

[{'children': [],
  'label': 'Totale generale delle Entrate',
  'slug': 'pcox-quadro-2-11',
  'values': [{'2016': {'abs': 6749319493.91, 'pc': 4993.71800473082}},
   {'2017': {'abs': 0.0, 'pc': 0.0}},
   {'2018': {'abs': 0.0, 'pc': 0.0}}]},
 {'children': [{'label': 'Imposte,tasse e proventi assimilati',
    'slug': 'pcox-quadro-2-2-10',
    'values': [{'2016': {'abs': 1534506629.01, 'pc': 1135.35792587391}},
     {'2017': {'abs': 0.0, 'pc': 0.0}},
     {'2018': {'abs': 0.0, 'pc': 0.0}}]},
   {'label': 'Compartecipazioni di tributi',
    'slug': 'pcox-quadro-2-2-12',
    'values': [{'2016': {'abs': 28476.0, 'pc': 0.0210689557711744}},
     {'2017': {'abs': 0.0, 'pc': 0.0}},
     {'2018': {'abs': 0.0, 'pc': 0.0}}]},
   {'label': 'Fondi perequativi  da Amministrazioni Centrali',
    'slug': 'pcox-quadro-2-2-14',
    'values': [{'2016': {'abs': 0.0, 'pc': 0.0}},
     {'2017': {'abs': 0.0, 'pc': 0.0}},
     {'2018': {'abs': 0.0, 'pc': 0.0}}]},
   {'label': 'Fondi perequativi  dalla Regione 

In [23]:
city_url

'https://www.openbilanci.it/armonizzati/bilanci/milano-comune-mi/entrate/dettaglio.json?year=2016&type=preventivo'